# Common patterns - mappable things

In [ ]:
:t map

In [ ]:
print $ map (2*) [1, 2, 3]

print $ map (\name -> "Hello, " ++ name ++ "!") ["Ann", "Bob"]

In [ ]:
data Option a = None | Some a deriving Show

In [ ]:
let a = Some 10

print a

In [ ]:
-- map :: forall a b. (a -> b) -> [a] -> [b]

optionMap :: (a -> b) -> Option a -> Option b
optionMap _ None = None
optionMap f (Some x) = Some (f x)


optionMap (*2) a

In [ ]:
data Tree a = Leaf | Node (Tree a) a (Tree a) deriving Show

tree = Node (Node Leaf 5 Leaf) 10 (Node Leaf 20 Leaf)

In [ ]:
-- map :: forall a b. (a -> b) -> [a] -> [b]

treeMap :: (a -> b) -> Tree a -> Tree b
treeMap _ Leaf = Leaf
treeMap f (Node left value right) = Node (treeMap f left) (f value) (treeMap f right)


treeMap (*2) tree

Is there a more general function that works for many different contexts? 

```haskell
type Functor :: (* -> *) -> Constraint

class Functor f where
    fmap :: (a -> b) -> f a -> f b
    (<$) :: a -> f b -> f a
    
    {-# MINIMAL fmap #-}
```

`fmap` takes an `a -> b` function and an `f a` data type (`a` wrapped in any context `f`). The function is applied to what's inside the context, and a value of type `b` wrapped in `f` is returned. The value can change (by applying the provided function), but the context remains the same.

In [ ]:
print $ fmap (*2) [1, 2, 3]

print $ fmap reverse ["hello", "world"]

In [ ]:
(*2) <$> [1, 2, 3]

reverse <$> ["hello", "world"]

In [ ]:
instance Functor Option where
    fmap _ None = None
    fmap g (Some x) = Some (g x)

In [ ]:
(*2) <$> Some 20

In [ ]:
instance Functor Tree where
    fmap _ Leaf = Leaf
    fmap g (Node left value right) = Node (fmap g left) (g value) (fmap g right)

In [ ]:
print $ fmap (*2) tree

The kind of Functor is `(* -> *) -> Constraint`, so we can implement Functor for types whose kind is `* -> *`, that is, something that can consume a type and returns a type. In other words, we can implement Functor for types that have one unapplied type variable.

We cannot implement Functor for `Int`, `String`, `Person`, etc. types (the kind of these things is `*`), but we can implement for `[]`, `Maybe`, `Tree`, etc. 

We can implement a Functor instance for a thing if it is
* `* -> *` (that is, `Type -> Type`)
* Functor laws

In [ ]:
:kind Tree

Some of the typeclasses have laws, that is, conditions that instances have to meet. These laws usually come from the math concept of the same name.


1. identity

`fmap id x == id x`


2. composition

`fmap f (fmap g x) == fmap (f . g) x` 